An overview of boosting tree algorithms, their main differences, performance comparisons, and hyperparameter optimization. In this notebook, we will delve deeper into boosted trees, specifically comparing XGBoost, CatBoost, and LightGBM. We will explore their main differences, parameters in each algorithm, compare their performance on different datasets, assess their CPU and GPU usage, conduct Optuna optimization, and examine SHAP values.

- [This Notebook repository](https://github.com/joaomh/xgboost-catboost-lgbm)
- [Undergraduate GitHub repository](https://github.com/joaomh/study_boosting_optuna_USP_undergraduate_thesis)
- [Link to my Undergraduate thesis in PT-BR](https://bdta.abcd.usp.br/item/003122385)


# Tables of Content:

**1. [Introduction](#Introduction)**

**2. [Ensemble Learning](#Ensemble)**

**3. [AdaBoost](#Ada)** 

**4. [GBMs](#GBM)** 

**5. [XGBoost vs. CatBoost vs. LightGBM](#XGBoost)** 
- 5.1 [Tree Symmetry](#Tree)
- 5.2 [Splitting Method](Splitting)

**6. [Models Performance CPU vs GPU](#Models)** 

**7. [Optuna in XGBoost vs. CatBoost vs. LightGBM](#Optuna)** 

**8. [Shap in XGBoost vs. CatBoost vs. LightGBM](#Shap)** 

**9. [Conclusions](#Conclusion)**

**10. [Bibliography](#Bibliography)**


# Introduction

The purpose of this post is to introduce the fundamentals of boosting algorithms and the main difference between XGBoost, CatBoost and LightGBM. We will give the reader some necessary keys to well understand and use related methods and be able to design adapted solutions when needed.

If we look at the [2022 Kaggle Data Science & ML Survey](https://www.kaggle.com/kaggle-survey-2022), we can see that Gradient Boosting Machines (GBMs) have been widely used in recent years. They are supervised machine learning algorithms that have consistently produced excellent results across a wide range of problems and have won numerous machine learning competitions.


![png1](./img/kaggle_state.png)

They achieve this because boosting algorithms are very effective on tabular datasets and offer the same performance as other state-of-the-art deep learning techniques, but they are easier to implement and cost less in terms of computer resources.

# Ensemble 
Many machine learning models primarily aim for high prediction accuracy using a single model, where boosting algorithms strive to enhance predictions by sequentially training a series of weak models, with each model compensating for the weaknesses of its predecessors.

First of all we need to understand Ensemble Learning, it's based on the idea of combining several simpler prediction models (weak learner), training them for the same task, and producing from them a more complex grouped model (strong learner) that is the sum of its parts.

For example, when creating an ensemble model based on several decision trees, which are simple yet high-variance models (often considered 'weak learners'), we need to aggregate them to enhance their resistance to data variations. Therefore, it makes sense to train the trees separately, allowing each one to adapt to different parts of the dataset. This way, each tree gains knowledge about various data variations, collectively improving the ensemble's predictive performance.

There are various ensemble learning methods, but in this text, we will primarily focus on Boosting, which is used in GBMs, but we can mention three algorithms that aims at combining weak learners:

**Bagging**: It is generally done with homogeneous predictors, each one operating independently in relation to the others, in a parallel manner. The final algorithm is then constructed by aggregating the results obtained from the base models in some form of average. Random Forest is one of the most famous algorithm.

**Boosting**: Generally implemented with homogeneous predictors, applied sequentially where the posterior model depends on the predecessor, and then these models are combined in the final ensemble. GBMs work like this

**Stacking**: It is typically done with heterogeneous predictors, training them in parallel, and then combining their outputs by training a meta-model that generates predictions based on the predictions of the various weak models. Here we can for combine RandomForest with DecisionTree for example.

![png1](./img/boosting_bagging.png) [Image from Ensemble Learning: Bagging & Boosting](https://towardsdatascience.com/ensemble-learning-bagging-boosting-3098079e5422)

# AdaBoost
AdaBoost is a specific Boosting algorithm developed for classification problems hte original AdaBoost algorithm is designed for classification problems, where the output is either −1 or 1, and the final prediction for a given instance is a weighted sum of each generated weak classifier

$$
G(x) = sign\bigr[\sum^M_{m=1}\alpha_m\cdot G_m(x)\bigr]
$$
Here, the weights $\alpha_m$
are computed by the boosting algorithm, and the idea is to increase the influence of weak learners that are more accurate while simultaneously penalizing those that are not.
The weakness is identified by the weak estimator error rate
$$err_m = \frac{\sum_{i=1}^Nw_i\mathbf{I}(y_i\neq G_m(x_i))}{\sum_{i=1}^Nw_i}$$


1. Initialize the observation weights $w_i = 1/N, i = 1, 2, . . . , N .$
2. For $m=1$ to $M$:

    2.1. Fit a classifier $G_m(x)$ to the training data using weights $w_i$

    2.2. Compute $err_m = \frac{\sum_{i=1}^Nw_i\mathbf{1}(y_i\neq G_m(x_i))}{\sum_{i=1}^Nw_i}$

    2.3. Compute $\alpha_m = log((1-err_m)/err_m)$

    2.4. Set $w_i \rightarrow w_i\cdot exp[\alpha_m \cdot \mathbf{1}(y_i\neq G_m(x_i))],i=1,2,...,N$

3. Output $G(x) = sign\bigr[\sum^M_{m=1}\alpha_m\cdot G_m(x)\bigr]$

From [1][2]

![png](img/ada.png) [Marsh, Brendan (2016). Multivariate Analysis of the Vector Boson Fusion Higgs Boson](https://www.researchgate.net/publication/306054843_Multivariate_Analysis_of_the_Vector_Boson_Fusion_Higgs_Boson)

Scikit-Learn have a implementation of AdaBoost

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X, y)
clf.predict([[0, 0, 0, 0]])
clf.score(X, y)

0.983

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an AdaBoostClassifier with a base DecisionTreeClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model
clf.score(X, y)

0.96

# GBMs
The Gradient Boosting Machines algorithm works by optimizing any given differentiable loss function, using gradient descent [3].

We can write de GBM model as 

$$F_M(x) = F_0(x) + \sum_{m=1}^MF_m(x)$$
$ \beta_mh(x; a_m)$ are the base functions learners, where $\beta_m$ is the weight, and $a_m$ the parameters of the learner $h$. And we have a loss function $L(y_i,F_m(x_i))$, so we would like to find all optimal values of this parameters that would minimize this loss funciton.
$$    \{\beta_m,\alpha_m\}_1^M = {\arg\min}_{\{\beta'_m,\alpha'_m\}_1^M}\sum_{i=1}^n L\Biggl(y^{(i)},\sum_{m=1}^M\beta'_mh(\mathbf{x}^{(i)};\alpha'_m)\Biggl)$$

In this situations where is infeasible we can try a 'greedy-stagewise' approach for $m=1,2,3,...,M$

$$(\beta_m,\alpha_m) = {\arg\min}_{\beta,\alpha}\sum_{i=1}^n L\Biggl(y^{(i)},F_{m-1}\mathbf{x}^{(i)} + \beta h(\mathbf{x}^{(i)};\alpha)\Biggl)$$
And then we can use a vectorized notation and make similar to the gradient descent formula. The learning rate, $\eta$ shrinks the influence of the new learner.
$$F_m(\mathbf{X}) = F_{m-1}(\mathbf{X}) + \eta \Delta_m(X)$$


The gradient of the loss function $L$ with relation to the last estimate $F_{m−1}(x)$ is,
$$-g_m(\mathbf{x}^{(i)}) = -\Bigg[\frac{\partial L(y^{(i)},c^{(i)})}{\partial F(\mathbf{x}^{(i)})}\Bigg]$$


Gradient of the loss function $L$ with respect to the last prediction is sometimes called pseudo-residual, and written as $r_{m−1}$ can be written as
$$\mathbf{r}_{m_1} = \nabla F_{m-1}(\mathbf{X})L(y,F_{m-1}(\mathbf{X})) = \nabla \hat{y}_{m-1}L(y,\hat{y}_{\mathbf{m-1}})$$


1. $F_0(\mathbf{X} = \arg\min_v\sum_{i=1}^n L(y^{(i)},v)$
2. For $m=1$ to $M$:

    2.1. $\mathbf{r}_{m_1} = \nabla \hat{y}_{m-1}L(y,\hat{y}_{\mathbf{m-1}})$ # Train a base learner minimizing squared error

    2.2. $\alpha = {\arg\min}_{\alpha,\beta}\sum_{i=1}^n(\mathbf{r}_{m-1}^{(i)}-\beta h(\mathbf{x}^{(i)};\alpha))^2$

    2.3. $\beta = {\arg\min}_{\beta}\sum_{i=1}^nL(y^{(i)},F_{m-1}(\mathbf{x}^{(i))}+\beta h(\mathbf{x}^{(i))};\alpha_m)$

    2.4. $\Delta_m(X) = \beta_mh(\mathbf{X};\alpha_m)$
    
    2.5 $F_m(\mathbf{X}) = F_{m-1}(\mathbf{X}) + \eta \Delta_m(X)$                                                              

3. Output $F_m$

From [3]


As you can see, it is an iterative algorithm that usually works with decision trees. We train a sequence of decision trees to gradually reduce the training error (each new tree tries to predict the residual error, this is the error at that current iteration and then we multiplied by the learning rate

![boost](img/boosting_tree.png)

As you can see the final prediction is:

initial_prediction + learning_rate*residual_0 + learning_rate*residual_1 _+ ... learning_rate*residual_N

Or

$F_m(\mathbf{X}) = F_{m-1}(\mathbf{X}) + \eta \Delta_m(X)$ 



We also can find Gradient Boosting function in scikit-learn

In [3]:
import pandas as pd
import numpy as np 
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
X = pd.DataFrame(load_breast_cancer()['data'],columns=load_breast_cancer()['feature_names'])
y = pd.DataFrame(load_breast_cancer()['target'],columns=['target'])
kf = KFold(n_splits=5,random_state=42,shuffle=True)
for train_index, index in kf.split(X):
    X_train,X_val = X.iloc[train_index],X.iloc[index],
    y_train,y_val = y.iloc[train_index],y.iloc[index],
gradient_booster = GradientBoostingClassifier()
gradient_booster.fit(X_train,y_train.values.ravel())
print(classification_report(y_val,gradient_booster.predict(X_val)))

              precision    recall  f1-score   support

           0       0.98      0.93      0.96        46
           1       0.96      0.99      0.97        67

    accuracy                           0.96       113
   macro avg       0.97      0.96      0.96       113
weighted avg       0.96      0.96      0.96       113



# XGBoost vs. CatBoost vs. LightGBM

XGBoost, Catboost, and LightGBM are all variations of gradient boosting algorithms, each employing decision trees as weak learners. I strongly recommend reading the papers [4], [5], [6]. Now, I'm going to highlight the main differences in each algorithm.

|   | XGBoost  | CatBoost  | LightGBM  |
|---|---|---|---|
|Developer |DMLC| Yandex| Microsoft|
|Release Year |2014 |2017 |2016|
|Tree Symmetry |Asymmetric: Level-wise tree growth |Symmetric |Asymmetric: Leaf-wise tree growth|
|Splitting Method |Pre-sorted and histogram-based algorithms |Greedy| GOSS|
|Categorical Columns |Support but must use numerical columns, cannot interpret ordinal category| Support |Support|
|Text Columns | Not Support| Support: Bag-of-Words, Naive-Bayes or BM25 to calculate numerical features from text| Not Support| 
|Missing Values | Handle | Handle |Handle|
|Training on|CPU and GPU|CPU and GPU|CPU and GPU|
|Others things| Works with Spark | Easy generate the Learning Curve |Have a RandomForest "boosting method"|

All of the models have different loss functions in their objectives, some of which are as follows:

For Regression:

* L2: mean squared error (default, recovers the mean value)
* L1: mean absolute error (good for outliers)
* MAPE: mean absolute percentage error (good for time series)
* Quantile: predict quantiles
* Poisson

For Classification:
* Logloss for binary classification
* Multiclass and cross-entropy for multi-class problems

For other loss functions, you can refer to the documentation of all three algorithms.


# Tree Symmetry
||||
|---|---|---|
![](https://github.com/joaomh/study_boosting_optuna_USP_undergraduate_thesis/blob/main/LaTeX-overleaf/images/CatBoost.png?raw=true)|![](https://github.com/joaomh/study_boosting_optuna_USP_undergraduate_thesis/blob/main/LaTeX-overleaf/images/XGboost.png?raw=true)|![](https://github.com/joaomh/study_boosting_optuna_USP_undergraduate_thesis/blob/main/LaTeX-overleaf/images/LGBM.png?raw=true)

CatBoost produces symmetric trees (or balanced trees). This refers to the splitting condition across all the nodes at the same depth. On the other hand, XGBoost and LightGBM produce asymmetric trees, meaning that the splitting condition at each node can be different.

Another important thing to note is that LightGBM grows leaf-wise (horizontally), while XGBoost grows level-wise (vertically). The picture below can show in more detail the differences in these growth types. This approach can lead to deeper trees with fewer nodes, potentially making it faster to train but may require more memory.

![](https://www.researchgate.net/publication/353155099/figure/fig2/AS:1044071766310913@1625937515739/Level-wise-vs-leaf-wise-tree-growth.png)

# Splitting Method
This determines how the splitting is determined in each algorithm.

In XGBoost, the pre-sorted algorithm considers all features and sorts them by value. The histogram algorithm groups feature values into discrete bins and finds the split point based on these bins. However, it is slower than GOSS.

CatBoost uses a greedy method where a list of possible candidates for feature splits is assigned to the leaf, and the split that results in the smallest penalty is selected.

In LightGBM, Gradient-based One-Side Sampling (GOSS) retains all the data with large gradients and performs random sampling for data instances with small gradients (small traning error). This results in fewer data instances used to train the model.

# Prevent Overfitting

All of the tree models come equipped with excellent parameters designed to mitigate overfitting. We will utilize many of these parameters in our Optuna hyperparameter optimization, some of them are:

**early_stopping_rounds:** This parameter employs an integer to halt the learning process. It identifies a point at which the validation score no longer improves, and in some cases, it may even start to deteriorate, while the training score continues to improve. This is not a hyperparameter that we intend to tune, but it's a crucial parameter to use, and it's not active by default.

**reg_alpha or lambda_l1:** These parameters represent the coefficient at the L1 regularization term of the cost function.

**reg_lambda or l2_leaf_reg:** These parameters represent the coefficient at the L2 regularization term of the cost function.

**learning_rate:** This setting is used to control the gradient step size and, in turn, affects the overall training time. Smaller values require more iterations for training.

**depth or max_depth:** This parameter limits the maximum depth of the tree model. It is employed to combat overfitting when dealing with small datasets.

**num_leaves or max_leaves:** The maximum number of leafs in the resulting tree

**random_strength:** This parameter determines the amount of randomness applied when scoring splits during the selection of the tree structure. You can adjust this parameter to mitigate the risk of overfitting in your model.

# Hyperparameter Tuning
As you can see all three libraries offer a variety of hyperparameters to tune, and their effectiveness can vary depending on the dataset. We will use Optuna in ours tests

# Models Performance CPU vs GPU

In this section, we are going to use three different datasets: epsilon, higgs, and breast cancer. However, we will not delve deeply into the typical steps of a data science project, such as EDA (Exploratory Data Analysis), pre-processing, handling missing values, plotting some variables, and analyzing correlations. Our primary focus will be the performance of out-of-the-box models, as they are designed to handle certain aspects by default, such as missing values.

## Epsilon
[Epsilon dataset](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#epsilon).
This dataset is best suited for binary classification.

The training dataset contains 400000 objects. Each object is described by 2001 columns. The first column contains the label value, all other columns contain numerical features.

The validation dataset contains 100000 objects. The structure is identical to the training dataset.


In [4]:
from catboost.datasets import epsilon
epsilon_train, epsilon_test = epsilon()

In [5]:
epsilon_train.shape, epsilon_test.shape

((400000, 2001), (100000, 2001))

In [6]:
# convert target -1 to 0
epsilon_train[epsilon_train[0] <= 0] = 0
epsilon_test[epsilon_test[0] <= 0] = 0

In [7]:
X_train = epsilon_train.loc[:,1:]
X_test = epsilon_test.loc[:,1:]
y_train = epsilon_train.loc[:,0]
y_test = epsilon_test.loc[:,0]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400000, 2000), (100000, 2000), (400000,), (100000,))

In [8]:
import timeit
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [9]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
time_cpu = ['CPU']
results_auc_cpu = []
models = [XGBClassifier(),
          CatBoostClassifier(verbose=False),
          LGBMClassifier(verbose=-1)]
for i in range(len(models)):
    start = timeit.default_timer()
    model_ = models[i].fit(X_train, y_train)
    stop = timeit.default_timer()
    time_cpu.append(stop-start)
    y_prob = model_.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    results_auc_cpu.append(metrics.auc(fpr, tpr))

In [10]:
time_cpu

['CPU', 82.3515075289979, 160.66510974999983, 26.330691929004388]

In [11]:
results_auc_cpu

[1.0, 1.0, 1.0]

In [12]:
models = [XGBClassifier(tree_method='gpu_hist'),
          CatBoostClassifier(verbose=False,task_type="GPU"),
          LGBMClassifier(verbose=-1,device='gpu')]

time_gpu = ['GPU']
results_auc_gpu = []
for i in range(len(models)):
    start = timeit.default_timer()
    model_ = models[i].fit(X_train, y_train)
    stop = timeit.default_timer()
    time_gpu.append(stop-start)
    y_prob = model_.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    results_auc_gpu.append(metrics.auc(fpr, tpr))

In [13]:
time_gpu

['GPU', 7.428800901994691, 17.33331783200265, 11.865153876002296]

In [14]:
results_auc_gpu

[1.0, 1.0, 1.0]

## Higgs

[Higgs](https://archive.ics.uci.edu/dataset/280/higgs)
This is a classification problem to distinguish between a signal process which produces Higgs bosons and a background process which does not. 

The training dataset contains 10500000 objects. Each object is described by 29 columns. The first column contains the label value, all other columns contain numerical features.

The validation dataset contains 5000000 objects. The structure is identical to the training dataset.
Method call format

In [15]:
from catboost.datasets import higgs
higgs_train, higgs_test = higgs()

In [16]:
higgs_train.shape, higgs_test.shape

((10500000, 29), (500000, 29))

In [17]:
X_train = higgs_train.loc[:,1:]
X_test = higgs_test.loc[:,1:]
y_train = higgs_train.loc[:,0]
y_test = higgs_test.loc[:,0]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10500000, 28), (500000, 28), (10500000,), (500000,))

In [18]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
time_cpu = ['CPU']
results_auc_cpu = []
models = [XGBClassifier(),
          CatBoostClassifier(verbose=False),
          LGBMClassifier(verbose=-1)]
for i in range(len(models)):
    start = timeit.default_timer()
    model_ = models[i].fit(X_train, y_train)
    stop = timeit.default_timer()
    time_cpu.append(stop-start)
    y_prob = model_.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    results_auc_cpu.append(metrics.auc(fpr, tpr))

In [19]:
time_cpu

['CPU', 175.2232390040008, 277.0851479319972, 15.447586549002153]

In [20]:
results_auc_cpu

[0.823429407021503, 0.8412804649425808, 0.8118326628507959]

In [21]:
models = [XGBClassifier(tree_method='gpu_hist'),
          CatBoostClassifier(verbose=False,task_type="GPU"),
          LGBMClassifier(verbose=-1,device='gpu')]

time_gpu = ['GPU']
results_auc_gpu = []
for i in range(len(models)):
    start = timeit.default_timer()
    model_ = models[i].fit(X_train, y_train)
    stop = timeit.default_timer()
    time_gpu.append(stop-start)
    y_prob = model_.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    results_auc_gpu.append(metrics.auc(fpr, tpr))

In [22]:
time_gpu

['GPU', 5.057594018006057, 39.93262233299902, 13.239043381996453]

In [23]:
results_auc_gpu

[0.8237744245413271, 0.8106215247755848, 0.8118326583877677]

## Breast Cancer

[Breast Cancer](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) 

The breast cancer dataset is a classic and very easy binary classification dataset.

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass.  They describe characteristics of the cell nuclei present in the image

In [24]:
from sklearn.datasets import load_breast_cancer
breast_cancer =  load_breast_cancer(as_frame=True).frame

In [25]:
from sklearn.model_selection import train_test_split
X = breast_cancer.drop(columns=['target'])
y = breast_cancer.loc[:,'target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

In [26]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
time_cpu = ['CPU']
results_auc_cpu = []
models = [XGBClassifier(),
          CatBoostClassifier(verbose=False),
          LGBMClassifier(verbose=-1)]
for i in range(len(models)):
    start = timeit.default_timer()
    model_ = models[i].fit(X_train, y_train)
    stop = timeit.default_timer()
    time_cpu.append(stop-start)
    y_prob = model_.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    results_auc_cpu.append(metrics.auc(fpr, tpr))

In [27]:
time_cpu

['CPU', 0.01815694000106305, 0.9057804709955235, 0.025702106999233365]

In [28]:
results_auc_cpu

[0.9947089947089948, 0.9972075249853027, 0.9944150499706055]

In [29]:
models = [XGBClassifier(tree_method='gpu_hist'),
          CatBoostClassifier(verbose=False,task_type="GPU"),
          LGBMClassifier(verbose=-1,device='gpu')]

time_gpu = ['GPU']
results_auc_gpu = []
for i in range(len(models)):
    start = timeit.default_timer()
    model_ = models[i].fit(X_train, y_train)
    stop = timeit.default_timer()
    time_gpu.append(stop-start)
    y_prob = model_.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    results_auc_gpu.append(metrics.auc(fpr, tpr))

In [30]:
time_gpu

['GPU', 0.05239353800425306, 16.459107301001495, 0.4470300850007334]

In [31]:
results_auc_gpu

[0.9938271604938271, 0.9977954144620811, 0.9954438565549677]

# Optuna

Now, let's attempt to utilize Optuna with the three algorithms, applying it to our largest dataset. We will employ early_stopping to determine the optimal number of iterations that minimizes the validation loss, and we'll also consider class weights using the 'balanced' option.

Another valuable aspect to explore is the use of sample weights, which can be passed as an array of shape n_samples. This feature proves exceptionally useful in applications such as churn modeling, where we aim to prevent the churn of high-value customers with greater profitability.

Here, we have a function that calculates numerous classification metrics. While our primary optimization focus will be on AUC, feel free to make adjustments as needed.

In [32]:
import sys
import timeit
import gc
from sklearn import metrics
import optuna
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lightgbm
from lightgbm import LGBMClassifier

def metrics_validation(y_test, y_prob):
    '''
    Input:
        y_prob: model predict prob
        y_test: target
    Output: Metrics of validation
        auc, ks, log_loss, accuracy
    '''
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(y_test, y_prob)
    ks = max(tpr - fpr) # Kolmogorov-Smirnov
    accu = metrics.accuracy_score(y_test, y_prob.round())
    precision = metrics.precision_score(y_test, y_prob.round()) # tp / (tp + fp)
    recall = metrics.recall_score(y_test, y_prob.round()) # tp / (tp + fn)
    f1_score = metrics.f1_score(y_test, y_prob.round()) # 2 * (precision * recall) / (precision + recall)
    return auc, log_loss, ks, 

Creating our objective function and the set of hyperparameter space."

In [33]:
def objective(trial, X_train, y_train, X_test, y_test, balanced, method):
    '''
    Input:
        trial: trial of the test
        X_train:
        y_train:
        X_test:
        y_test:
        balanced:balanced or None
        method: XGBoost, CatBoost or LGBM
    Output: Metrics of validation
        auc, ks, log_loss
        metrics_validation(y_test, y_pred)[0]
    '''
    gc.collect()
    if method=='LGBM':
        param_grid = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True),
                      'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                      'lambda_l1': trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
                      'lambda_l2': trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
                      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
                      'max_depth': trial.suggest_int('max_depth', 5, 64),
                      'feature_fraction': trial.suggest_float("feature_fraction", 0.4, 1.0),
                      'bagging_fraction': trial.suggest_float("bagging_fraction", 0.4, 1.0),
                      'device':'gpu',
                      'bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
  
                     }
        model = LGBMClassifier(**param_grid)

        print('LGBM - Optimization using optuna')
        model.fit(X_train, y_train)
        
        y_pred = model.predict_proba(X_test)[:,1]

    if method=='CATBoost':
        param_grid = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True),
                      'depth': trial.suggest_int("depth", 4, 10),
                      'max_bin': trial.suggest_int('max_bin', 200, 400),
                      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
                      'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10, log = True),
                      'random_seed': 42,
                      'random_strength': trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
                      'bagging_temperature': trial.suggest_float("bagging_temperature", 0.0, 10.0),
                      'od_type': trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
                      'task_type':'GPU',
                      'od_wait': trial.suggest_int("od_wait", 10, 50),
                     }
        if len(X_train._get_numeric_data().columns) != len(X_train.columns):
            categorical_features_indices = list(X_train.select_dtypes(exclude='number').columns)
            model = CatBoostClassifier(**param_grid)
            print('CATBoost - Optimization using optuna')
            model.fit(X_train, y_train,cat_features=categorical_features_indices,verbose=False)
            y_pred = model.predict_proba(X_test)[:,1]
        else:
            model = CatBoostClassifier(**param_grid)
            print('CATBoost - Optimization using optuna')
            model.fit(X_train, y_train,verbose=False)
            y_pred = model.predict_proba(X_test)[:,1]
        
    if method=='XGBoost':
        param_grid = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True),
                      'max_depth': trial.suggest_int('max_depth', 3, 16),
                      'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
                      'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log = True),
                      'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),
                      'lambda': trial.suggest_float('lambda', 0.0001, 10.0, log = True),
                      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.8),
                      'booster': 'gbtree',
                      'tree_method':'gpu_hist',
                      'random_state': 42,
                     }
        model = XGBClassifier(**param_grid)
        print('XGBoost - Optimization using optuna')
        model.fit(X_train, y_train,verbose=False)
        y_pred = model.predict_proba(X_test)[:,1]
    
    auc_res, log_loss_res, ks_res = metrics_validation(y_test, y_pred)
    print('auc:'+str(auc_res),', log_loss:'+str(log_loss_res),', ks:'+str(ks_res))
    return metrics_validation(y_test, y_pred)[0]

Tuning the model: Here, the study will be created, and an important aspect to note is the **time_max_tuning,** which represents the maximum time in seconds to stop.

In [37]:
def tuning(X_train, y_train, X_test, y_test, balanced, method):
    '''
    Input:
        trial: 
        x_train:
        y_train:
        X_test:
        y_test:
        balanced:balanced or not balanced
        method: XGBoost, CatBoost or LGBM
    Output: Metrics of validation
        auc, ks, log_loss
        metrics_validation(y_test, y_pred)[0]
    '''
    study = optuna.create_study(direction='maximize', study_name=method+' Classifier')
    func = lambda trial: objective(trial, X_train, y_train, X_test, y_test, balanced, method)
    print('Starting the optimization')
    time_max_tuning = 15*60 # max time in seconds to stop
    study.optimize(func, timeout=time_max_tuning)
    return study

Train the model while implementing **early_stopping**, and then return the best model.

In [35]:
def train(X_train, y_train, X_test, y_test, balanced, method):
    '''
    Input:
        X_train:
        y_train:
        X_test:
        y_test:
        balanced:balanced or None
        method: XGBoost, CatBoost or LGBM
    Output: predict model
    '''
    print('Tuning')
    study = tuning(X_train, y_train, X_test, y_test, balanced, method)
    if method=='LGBM':
        model = LGBMClassifier(**study.best_params)
        print('Last Fit')
        model.fit(X_train, y_train, eval_set=[(X_test,y_test)],
                 callbacks = [lightgbm.early_stopping(stopping_rounds=100), lightgbm.log_evaluation(period=5000)])
    if method=='CATBoost':
        model = CatBoostClassifier(**study.best_params)
        if len(X_train._get_numeric_data().columns) != len(X_train.columns):
            categorical_features_indices = list(X_train.select_dtypes(exclude='number').columns)
            print('Last Fit')
            model.fit(X_train, y_train,cat_features=categorical_features_indices, eval_set=[(X_test,y_test)],
                 early_stopping_rounds=100,verbose = False)
        else:
            print('Last Fit')
            model.fit(X_train, y_train, eval_set=[(X_test,y_test)],
                 early_stopping_rounds=100,verbose = False)
    if method=='XGBoost':
        model = XGBClassifier(**study.best_params)
        print('Last Fit')
        model.fit(X_train, y_train, eval_set=[(X_test,y_test)],
                 early_stopping_rounds=100,verbose = False)
    return model, study

In [36]:
X_train = higgs_train.loc[:,1:]
X_test = higgs_test.loc[:,1:]
y_train = higgs_train.loc[:,0]
y_test = higgs_test.loc[:,0]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10500000, 28), (500000, 28), (10500000,), (500000,))

In [38]:
xgb_model, study_lgbm = train(X_train, y_train, X_test, y_test, balanced='balanced', method='XGBoost')

[I 2023-10-02 18:19:46,797] A new study created in memory with name: XGBoost Classifier


Tuning
Starting the optimization
XGBoost - Optimization using optuna
auc:0.7823937789568013 , log_loss:0.5783660516621543 , ks:0.4161117447746004


[I 2023-10-02 18:19:54,363] Trial 0 finished with value: 0.7823937789568013 and parameters: {'learning_rate': 0.05371518813357488, 'max_depth': 5, 'min_child_weight': 175, 'gamma': 0.0035416020979473044, 'alpha': 1.8148619430935994e-05, 'lambda': 0.0004628148705354307, 'colsample_bytree': 0.24118386631381142}. Best is trial 0 with value: 0.7823937789568013.


XGBoost - Optimization using optuna
auc:0.7642073236693494 , log_loss:0.6704264957156165 , ks:0.38831003181042867


[I 2023-10-02 18:20:01,994] Trial 1 finished with value: 0.7642073236693494 and parameters: {'learning_rate': 0.0022732346204839453, 'max_depth': 5, 'min_child_weight': 22, 'gamma': 0.0009470349674428265, 'alpha': 1.1868156471657653e-07, 'lambda': 0.6074789905304752, 'colsample_bytree': 0.5526828609424803}. Best is trial 0 with value: 0.7823937789568013.


...

XGBoost - Optimization using optuna
auc:0.8299277506251002 , log_loss:0.5164700785104678 , ks:0.4940057310612048


[I 2023-10-02 18:34:42,509] Trial 49 finished with value: 0.8299277506251002 and parameters: {'learning_rate': 0.02904432582357365, 'max_depth': 14, 'min_child_weight': 26, 'gamma': 0.00022515439419978954, 'alpha': 0.00027383846560024666, 'lambda': 0.14731548821038506, 'colsample_bytree': 0.7108260793389256}. Best is trial 28 with value: 0.8365957634913157.


XGBoost - Optimization using optuna
auc:0.8320791374156545 , log_loss:0.5038430738703974 , ks:0.498138605830747


[I 2023-10-02 18:34:55,471] Trial 50 finished with value: 0.8320791374156545 and parameters: {'learning_rate': 0.07280910837799989, 'max_depth': 12, 'min_child_weight': 84, 'gamma': 0.0012198177812870255, 'alpha': 0.00044684863879203296, 'lambda': 0.1355095220732296, 'colsample_bytree': 0.556349011421445}. Best is trial 28 with value: 0.8365957634913157.
/home/joao/.local/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Last Fit


In [39]:
cat_model, study_lgbm = train(X_train, y_train, X_test, y_test, balanced='balanced', method='CATBoost')

[I 2023-10-02 18:38:59,284] A new study created in memory with name: CATBoost Classifier


Tuning
Starting the optimization
CATBoost - Optimization using optuna
auc:0.7998754143643698 , log_loss:0.5452211683603745 , ks:0.4438787464411975


[I 2023-10-02 18:40:23,579] Trial 0 finished with value: 0.7998754143643698 and parameters: {'learning_rate': 0.015164908470991641, 'depth': 10, 'max_bin': 294, 'min_data_in_leaf': 47, 'l2_leaf_reg': 0.9145165239547186, 'random_strength': 1.531782669160816e-07, 'bagging_temperature': 8.333557651780637, 'od_type': 'Iter', 'od_wait': 25}. Best is trial 0 with value: 0.7998754143643698.


CATBoost - Optimization using optuna
auc:0.7545641522481075 , log_loss:0.6432285925227534 , ks:0.3692005619994447


[I 2023-10-02 18:41:12,781] Trial 1 finished with value: 0.7545641522481075 and parameters: {'learning_rate': 0.0004159918549688078, 'depth': 7, 'max_bin': 339, 'min_data_in_leaf': 11, 'l2_leaf_reg': 0.010019575051730078, 'random_strength': 0.011947716010460267, 'bagging_temperature': 7.60686171918655, 'od_type': 'Iter', 'od_wait': 11}. Best is trial 0 with value: 0.7998754143643698.


...

CATBoost - Optimization using optuna
auc:0.8319333098981236 , log_loss:0.5018822542548416 , ks:0.49768661286598603


[I 2023-10-02 18:53:52,027] Trial 15 finished with value: 0.8319333098981236 and parameters: {'learning_rate': 0.09777944001405149, 'depth': 8, 'max_bin': 207, 'min_data_in_leaf': 80, 'l2_leaf_reg': 0.00021496491667450296, 'random_strength': 1.1395309760736173e-08, 'bagging_temperature': 3.5176935452677727, 'od_type': 'Iter', 'od_wait': 31}. Best is trial 11 with value: 0.8347870809351778.


CATBoost - Optimization using optuna
auc:0.8118436357833898 , log_loss:0.5289626734569585 , ks:0.46263545659866717


[I 2023-10-02 18:55:05,433] Trial 16 finished with value: 0.8118436357833898 and parameters: {'learning_rate': 0.009377081210354962, 'depth': 9, 'max_bin': 321, 'min_data_in_leaf': 179, 'l2_leaf_reg': 0.00010611211598248217, 'random_strength': 7.326450123117985e-06, 'bagging_temperature': 2.4954379330081213, 'od_type': 'Iter', 'od_wait': 43}. Best is trial 11 with value: 0.8347870809351778.


Last Fit


In [40]:
lgbm_model, study_lgbm = train(X_train, y_train, X_test, y_test, balanced='balanced', method='LGBM')

[I 2023-10-02 18:57:19,240] A new study created in memory with name: LGBM Classifier


Tuning
Starting the optimization
LGBM - Optimization using optuna
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] feature_fraction is set=0.6035724807065478, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6035724807065478
[LightGBM] [Warning] lambda_l2 is set=1.1747600378139125, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1747600378139125
[LightGBM] [Warning] lambda_l1 is set=2.1027251445636707e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.1027251445636707e-08
[LightGBM] [Warning] bagging_fraction is set=0.8944699443091366, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8944699443091366
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[Light

auc:0.7898983089924385 , log_loss:0.6848058696956106 , ks:0.42700184528429935


[I 2023-10-02 18:57:47,245] Trial 0 finished with value: 0.7898983089924385 and parameters: {'learning_rate': 0.0004577801504792331, 'num_leaves': 173, 'lambda_l1': 2.1027251445636707e-08, 'lambda_l2': 1.1747600378139125, 'min_data_in_leaf': 12, 'max_depth': 31, 'feature_fraction': 0.6035724807065478, 'bagging_fraction': 0.8944699443091366, 'bagging_freq': 3}. Best is trial 0 with value: 0.7898983089924385.


LGBM - Optimization using optuna
[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] feature_fraction is set=0.8550777091007526, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8550777091007526
[LightGBM] [Warning] lambda_l2 is set=0.0001164532088766597, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001164532088766597
[LightGBM] [Warning] lambda_l1 is set=3.3992066902339477e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.3992066902339477e-07
[LightGBM] [Warning] bagging_fraction is set=0.9631456557452717, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9631456557452717
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] feature_frac

...

LGBM - Optimization using optuna
[LightGBM] [Warning] min_data_in_leaf is set=92, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=92
[LightGBM] [Warning] feature_fraction is set=0.6432197846294514, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6432197846294514
[LightGBM] [Warning] lambda_l2 is set=3.625092429340071e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.625092429340071e-07
[LightGBM] [Warning] lambda_l1 is set=0.004833154954941355, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.004833154954941355
[LightGBM] [Warning] bagging_fraction is set=0.5977756667534049, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5977756667534049
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=92, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=92
[LightGBM] [Warning] feature_fraction

[I 2023-10-02 19:12:03,340] Trial 38 finished with value: 0.8106298257510343 and parameters: {'learning_rate': 0.029456506763463605, 'num_leaves': 192, 'lambda_l1': 0.004833154954941355, 'lambda_l2': 3.625092429340071e-07, 'min_data_in_leaf': 92, 'max_depth': 41, 'feature_fraction': 0.6432197846294514, 'bagging_fraction': 0.5977756667534049, 'bagging_freq': 5}. Best is trial 27 with value: 0.828677088494562.


LGBM - Optimization using optuna
[LightGBM] [Warning] min_data_in_leaf is set=64, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=64
[LightGBM] [Warning] feature_fraction is set=0.6136836521982938, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6136836521982938
[LightGBM] [Warning] lambda_l2 is set=4.3850314935049856e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.3850314935049856e-05
[LightGBM] [Warning] lambda_l1 is set=7.070420118340646e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.070420118340646e-05
[LightGBM] [Warning] bagging_fraction is set=0.6224834798074727, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6224834798074727
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] min_data_in_leaf is set=64, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=64
[LightGBM] [Warning] feature_frac

[I 2023-10-02 19:12:24,868] Trial 39 finished with value: 0.8019253626823509 and parameters: {'learning_rate': 0.01631914456021541, 'num_leaves': 169, 'lambda_l1': 7.070420118340646e-05, 'lambda_l2': 4.3850314935049856e-05, 'min_data_in_leaf': 64, 'max_depth': 31, 'feature_fraction': 0.6136836521982938, 'bagging_fraction': 0.6224834798074727, 'bagging_freq': 4}. Best is trial 27 with value: 0.828677088494562.


Last Fit
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] feature_fraction is set=0.6363259048680269, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6363259048680269
[LightGBM] [Warning] lambda_l2 is set=6.657688122881207e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.657688122881207e-06
[LightGBM] [Warning] lambda_l1 is set=0.0014235559343150825, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0014235559343150825
[LightGBM] [Warning] bagging_fraction is set=0.6770301860680766, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6770301860680766
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] feature_fraction is set=0.636325904868

AUC in the final models

||XGBoost|CatBoost|LightGBM|
|---|---|---|---|
|Number of Trials|50|16|39|
|AUC|0.8365957634913157|0.8347870809351778|0.828677088494562|

## Final hyperparameters in our models

### XGBoost
```
Trial 28 finished with value: 0.8365957634913157 and parameters: {'learning_rate': 0.08684579563106676, 'max_depth': 15, 'min_child_weight': 197, 'gamma': 0.0184790670508487, 'alpha': 3.800980410024799e-08, 'lambda': 0.8947997949504739, 'colsample_bytree': 0.5074143246162668}. Best is trial 28 with value: 0.8365957634913157.
```
### CatBoost
```
Trial 11 finished with value: 0.8347870809351778 and parameters: {'learning_rate': 0.09193395513931629, 'depth': 8, 'max_bin': 292, 'min_data_in_leaf': 94, 'l2_leaf_reg': 2.4060335322115672e-05, 'random_strength': 1.117086010478237e-05, 'bagging_temperature': 2.8645589742944164, 'od_type': 'Iter', 'od_wait': 39}. Best is trial 11 with value: 0.8347870809351778.
```
### LightGBM
```
Trial 27 finished with value: 0.828677088494562 and parameters: {'learning_rate': 0.0979876550122422, 'num_leaves': 224, 'lambda_l1': 0.0014235559343150825, 'lambda_l2': 6.657688122881207e-06, 'min_data_in_leaf': 81, 'max_depth': 49, 'feature_fraction': 0.6363259048680269, 'bagging_fraction': 0.6770301860680766, 'bagging_freq': 3}. Best is trial 27 with value: 0.828677088494562.
```

In [ ]:
lgbm_model

# SHAP in XGBoost, CatBoost, LightGBM

## SHAP using CPU

In [ ]:
import shap
time_cpu_shap = ['CPU']
start = timeit.default_timer()
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_train)
stop = timeit.default_timer()
time_cpu_shap.append(stop-start)
shap.summary_plot(shap_values[1], X_train,show=False)

In [ ]:
import shap
time_cpu_shap = ['CPU']
start = timeit.default_timer()
explainer = shap.TreeExplainer(cat_model)
shap_values = explainer.shap_values(X_train)
stop = timeit.default_timer()
time_cpu_shap.append(stop-start)
shap.summary_plot(shap_values[1], X_train,show=False)

In [ ]:
import shap
time_cpu_shap = ['CPU']
start = timeit.default_timer()
explainer = shap.TreeExplainer(lgbm_model)
shap_values = explainer.shap_values(X_train)
stop = timeit.default_timer()
time_cpu_shap.append(stop-start)
shap.summary_plot(shap_values[1], X_train,show=False)

In [ ]:
time_cpu_shap

## SHAP using GPU

In [ ]:
time_gpu_shap = ['CPU']
start = timeit.default_timer()
explainer = shap.explainers.GPUTree(xgb_model)
stop = timeit.default_timer()
time_cpu_shap.append(stop-start)
shap.summary_plot(shap_values[1], X_train,show=False)

In [ ]:
time_gpu_shap = ['CPU']
start = timeit.default_timer()
explainer = shap.explainers.GPUTree(cat_model)
stop = timeit.default_timer()
time_cpu_shap.append(stop-start)
shap.summary_plot(shap_values[1], X_train,show=False)

In [ ]:
time_gpu_shap = ['CPU']
start = timeit.default_timer()
explainer = shap.explainers.GPUTree(lgbm_model)
stop = timeit.default_timer()
time_cpu_shap.append(stop-start)
shap.summary_plot(shap_values[1], X_train,show=False)

In [ ]:
time_cpu_shap

# Bibliography
You can find all files in [this repository](https://github.com/joaomh/xgboost-catboost-lgbm)

References, and, of course, you can access the documentation for each algorithm. 


[1] - [Schapire, Robert E(1999). A Short Introduction to Boosting](https://cseweb.ucsd.edu/~yfreund/papers/IntroToBoosting.pdf)

[2] - [HASTIE, T.; TIBSHIRANI, R.; FRIEDMAN, J. (2009). The Elements of Statistical Learning](https://hastie.su.domains/Papers/ESLII.pdf)

[3] - [Jerome H. Friedman (2001). GREEDY FUNCTION APPROXIMATION:A GRADIENT BOOSTING MACHINE](https://jerryfriedman.su.domains/ftp/trebst.pdf)

[4] - [Tianqi Chen, Carlos Guestrin (2016).XGBoost: {A} Scalable Tree Boosting System](https://arxiv.org/abs/1603.02754)

[5] - [Anna Veronika Dorogush, Andrey Gulin, Gleb Gusev, Nikita Kazeev, Liudmila Ostroumova Prokhorenkova, Aleksandr Vorobev (2017).CatBoost: unbiased boosting with categorical features](https://arxiv.org/abs/1706.09516)

[6] - [Ke, Guolin and Meng, Qi and Finley, Thomas and Wang, Taifeng and Chen, Wei and Ma, Weidong and Ye, Qiwei and Liu, Tie-Yan (2017).Lightgbm: A highly efficient gradient boosting decision tree](https://proceedings.neurips.cc/paper_files/paper/2017/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf)

[7] - [Anna Veronika Dorogush: Mastering gradient boosting with CatBoost | PyData London 2019](https://www.youtube.com/watch?v=usdEWSDisS0)

[8] - [Pedro Tabacof - Unlocking the Power of Gradient-Boosted Trees (using LightGBM) | PyData London 2022](https://www.youtube.com/watch?v=qGsHlvE8KZM)

[9] - [Pinheiro, J., & Becker, M.. (2023). Um estudo sobre algoritmos de Boosting e a otimização de hiperparâmetros utilizando optuna.](https://bdta.abcd.usp.br/item/003122385)